In [ ]:
!python3 --version

In [ ]:
!python3 -m pip --version

In [ ]:
!python3 -m pip install -r requirements/dev.txt

In [ ]:
!ls data/UGallery -sh

# PyTorch DataLoaders

In [ ]:
# import json
import time
# from collections import Counter, defaultdict
from copy import deepcopy
from os.path import join

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from tqdm.notebook import tqdm

from datasets import UGalleryDataset
from models import CuratorNet
from samplers import SameProfileSizeBatchSampler
from samplers.utils import merge_samples
from transforms import DictToTensor
from utils.ugallery.data import load_embedding

## Loading data

In [ ]:
SETTINGS = {
    "dataloader:batch_size": 4096 * 3,
    "dataloader:num_workers": 4,
    "training:num_epochs": 150,
    "optimizer:lr": 0.0001,
    "optimizer:weight_decay": 0.001,
    "scheduler:factor": 0.6,
}
# NOTE(Antonio): Double learning rate if you double batch size.

In [ ]:
# Load embeddings from the available files
resnet50_embedding_path = join("data", "UGallery", "ugallery_resnet50_embeddings.npy")
embedding = load_embedding(resnet50_embedding_path)

## Setup training

In [ ]:
# Model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CuratorNet()
model = model.to(device)

In [ ]:
# Training criteria
optimizer = optim.Adam(model.parameters(), lr=SETTINGS["optimizer:lr"], weight_decay=SETTINGS["optimizer:weight_decay"])
criterion = nn.BCEWithLogitsLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=SETTINGS["scheduler:factor"], patience=2, verbose=True)

In [ ]:
# Dataloaders (training)
training_dataset = UGalleryDataset(
    csv_file="data/UGallery/train_public.csv",
    embedding=embedding.features,
    transform=transforms.Compose([
        DictToTensor(),
    ]))
training_dataset.prepare(embedding.id2index)
training_sampler = RandomSampler(training_dataset)
training_batch_sampler = SameProfileSizeBatchSampler(sampler=training_sampler, batch_size=SETTINGS["dataloader:batch_size"])
training_dataloader = DataLoader(training_dataset, collate_fn=merge_samples, batch_sampler=training_batch_sampler, num_workers=SETTINGS["dataloader:num_workers"], pin_memory=True)

In [ ]:
# Dataloaders (validation)
validation_dataset = UGalleryDataset(
    csv_file="data/UGallery/validation_public.csv",
    embedding=embedding.features,
    transform=transforms.Compose([
        DictToTensor(),
    ]))
validation_dataset.prepare(embedding.id2index)
validation_sampler = SequentialSampler(validation_dataset)
validation_batch_sampler = SameProfileSizeBatchSampler(sampler=validation_sampler, bump_rate=0.0, batch_size=SETTINGS["dataloader:batch_size"])
validation_dataloader = DataLoader(validation_dataset, collate_fn=merge_samples, batch_sampler=validation_batch_sampler, num_workers=SETTINGS["dataloader:num_workers"], pin_memory=True)

## Define training procedure

In [ ]:
def train_model(model, device, criterion, optimizer, scheduler, dataloaders, num_epochs=1, summary_writer=None):
    model = model.to(device)
    start = time.time()
    best_model_wts = deepcopy(model.state_dict())
    best_validation_acc = 0.0
    # Checkpoint
    checkpoint_filename = f"{model.__class__.__name__}_{time.strftime('%Y-%m-%d-%H-%M-%S')}"
    checkpoint_filepath = join("checkpoints", checkpoint_filename)
    print(f"Checkpoints stored at {checkpoint_filepath}")
    
    for epoch in range(1, num_epochs + 1):
        print(f"Epoch {epoch}/{num_epochs}")
        
        # Each epoch has a training and validation phase
        for phase in ["train", "validation"]:
            if phase == "train":
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_acc = 0
            running_x = 0
                
            # Iterate over data
            for i_batch, (batch, target) in enumerate(tqdm(dataloaders[phase], desc=f"Epoch {epoch} ({phase})")):
                batch = {
                    k: v.to(device)
                    for k, v in batch.items()
                }
                
                target = target.to(device)
                
                # Restart params gradients
                optimizer.zero_grad()
                
                # Forward pass
                with torch.set_grad_enabled(phase == "train"):
                    output = model(**batch)
                    loss = criterion(output, target)
                    # Backward pass
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                
                # Statistics
                running_loss += loss.item() * output.size(0)
                running_acc += (output.cpu().detach().numpy() > 0).sum()
                running_x += output.size(0)
                
                if i_batch % 40 == 39 and summary_writer:
                    summary_writer.add_scalar(
                        f"{phase} loss",
                        running_loss / running_x,
                        (epoch - 1) * len(dataloaders[phase]) + i_batch,
                    )
                    summary_writer.close()
            
            # Logging
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_acc / len(dataloaders[phase].dataset)
            print(f"{phase.title()} loss: {epoch_loss}")
            print(f"{phase.title()} acc = {100 * epoch_acc}%")
            
            # Deepcopy if model is good
            if phase == "validation" and epoch_acc > best_validation_acc:
                print(f"New best model with ~{round(100 * epoch_acc, 4)}% acc ({epoch_acc})")
                best_validation_acc = epoch_acc
                best_epoch = scheduler.last_epoch
                best_model_wts = deepcopy(model.state_dict())
                torch.save({
                    "epoch": best_epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "scheduler_state_dict": scheduler.state_dict(),
                    "validation_accuracy": best_validation_acc,    
                }, checkpoint_filepath)
                print(f"Saved model at {checkpoint_filepath}")
            
            # Scheduler step if necessary
            if phase == "validation":
                print(f"Scheduler: {scheduler.num_bad_epochs} bad epoch(s) (patience={scheduler.patience})")
                scheduler.step(epoch_acc)

        print()
    
    elapsed = time.time() - start
    print(f"Training completed in {elapsed // 60:.0f}m {elapsed % 60:.0f}s")
    print(f"Best validation accuracy: ~{round(100 * best_validation_acc, 4)}%")
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_validation_acc, best_epoch

## Training model

In [ ]:
# SummaryWriter to track training progress
summary_writer_name = "CuratorNet_UGallery_" + "_".join([f"{k}={v}" for k, v in SETTINGS.items()])
summary_writer = SummaryWriter(f"runs/{summary_writer_name}", flush_secs=20)

In [ ]:
# Apply training procedure
validation_accuracy = 0.0
best_epoch = 0
model, validation_accuracy, best_epoch = train_model(
    model, device,
    criterion, optimizer, scheduler,
    {"train": training_dataloader, "validation": validation_dataloader},
    # num_epochs=SETTINGS["training:num_epochs"],
    summary_writer=summary_writer,
)

In [ ]:
# Save last model
checkpoint_filename = f"{model.__class__.__name__}_{time.strftime('%Y-%m-%d-%H-%M-%S')}"
torch.save({
    "best_epoch": best_epoch,
    "epoch": scheduler.last_epoch,
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "scheduler_state_dict": scheduler.state_dict(),
    "validation_accuracy": validation_accuracy,    
}, join("checkpoints", checkpoint_filename))
print("Saved model")